# Data Encryption Standard (DES)

___________

DES es el primer cifrado estandarizado por una institución oficial. Se trata de un esfuerzo por crear un criptosistema seguro para ser aceptado de manera amplia, oficial e internacional. DES fue un método muy utilizado por su seguridad desde el diseño. Eventualmente fue reemplazado por AES, una versión más avanzada y segura de un cifrado de llave privada por bloques.

Es un criptosistema que toma bloques de 64 bits y devuelve bloques de 64 bits. De tener un mensaje que no consista en un múltiplo de 64 bits, se debe rellenar con bits elegidos al azar o por el usuario hasta llegar a un múltiplo de 64 bits. También se necesita una llave de 64 bits. Aunque técnicamente se trate de una de 56 bits con 8 bits de paridad.

## Manejo de subllaves

A partir de la llave original debemos obtener 16 subllaves de 48 bits para usar posteriormente en el algoritmo. Como ya se había mencionado hay 8 bits de paridad, específicamente los bits en posiciones múltiplos de 8 en la llave. Los 56 bits restantes son permutados con una función llamada $PC-1$.

![PC-1](./img/PC-1.png)

Es decir el bit 57 va al primer lugar, el bit 49 va al segundo, etc. Luego dividimos el resultado en dos partes de 28 bits, tomando los primeros 28 y llamándolo $C_0$ y los últimos 28 y llamándolo $D_0$. Iteramos los siguientes $C_i's$ y $D_i's$ como $C_i=LS_i(C_{i-1}))$ y $D_i=LS_i(D_{i-1}))$ con $LS_i$ una función que dependiendo de la $i$ toma el último o dos últimos bits y los mueve al principio.

|i|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|**bits**|1|1|2|2|2|2|2|2|1|2|2|2|2|2|2|1|

A partir de estas $C's$ y $D's$ se obtienen las 16 subllaves $K_i$ concatenando $D_i$ luego de $C_i$ y permutando 48 de sus bits de acuerdo a una función $PC-2$. Es decir que $K_i=PC-2(C_iD_i)$.

![PC-2](./img/PC-2.png)

Probemos la creación de subllaves en python:

In [1]:
k = b"mi llave" #elegimos una llave de 8 bytes o letras

pc1 = [56, 48, 40, 32, 24, 16,  8,
  0, 57, 49, 41, 33, 25, 17,
  9,  1, 58, 50, 42, 34, 26,
 18, 10,  2, 59, 51, 43, 35,
 62, 54, 46, 38, 30, 22, 14,
  6, 61, 53, 45, 37, 29, 21,
 13,  5, 60, 52, 44, 36, 28,
 20, 12,  4, 27, 19, 11,  3]

left_rotations = [1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]

pc2 = [
13, 16, 10, 23,  0,  4,
 2, 27, 14,  5, 20,  9,
22, 18, 11,  3, 25,  7,
15,  6, 26, 19, 12,  1,
40, 51, 30, 36, 46, 54,
29, 39, 50, 44, 32, 47,
43, 48, 38, 55, 33, 52,
45, 41, 49, 35, 28, 31]

l = len(k) * 8 #convertimos la llave a un arreglo de bits
K = [0] * l
pos = 0
for ch in k:
	i = 7
	while i >= 0:
		if ch & (1 << i) != 0:
			K[pos] = 1
		else:
			K[pos] = 0
		pos += 1
		i -= 1

Ahora permutamos con $PC-1$ y dividimos en dos partes.

In [2]:
k0 = list(map(lambda x: K[x], pc1))

C = k0[:28]
D = k0[28:]

Ahora iteramos los $C_i's$ y $D_i's$ obteniendo las 16 subllaves en el arreglo Kn.

In [3]:
Kn = [0]*16
for i in range(16):
    for j in range(left_rotations[i]): #rotación izquierda de acuerdo a la i
        C.append(C[0])
        del C[0]
        D.append(D[0])
        del D[0]
    CD = C + D #concatenación
    Kn[i] = list(map(lambda x: CD[x], pc2)) #aplicación de la permutación

## Cifrado

Como se había mencionado, primero se añade los bits suficientes para que el mensaje tenga un número de bits múltiplo de 64. Veremos lo que le ocurre a cada bloque de 64 bits.

El primer paso es una permutación inicial llamada $IP$ y luego se divide el resultado en los primeros 32 bits nombrados $L_0$ y los últimos 32 bits nombrados $R_0$. Esto significa que $IP(m)=L_0R_0$.

![IP](./img/IP.png)

Después iteramos para obtener los siguientes 16 $L_i's$ y $R_i's$. El proceso es por un lado $L_i=R_{i-1}$ y por otro $R_i=L_{i-1}\oplus f(R_{i-1},K_i)$, donde $\oplus$ representa la operación $XOR$ bit a bit y $f$ es una función llamada de Feistel que será discutida a detalle. La última pareja se concatena de manera opuesta a la inicial y se hace pasar por la permutación inversa a la inicial para obtener el mensaje encriptado. Es decir que $c=IP^{-1}(R_{16}L_{16})$

### Feistel

La función f de Feistel toma primero el bloque $R_{i-1}$ y lo expande a ser de 48 bits a través de una permutación que repite algunos de sus elementos. Esta permutación se llama $E$.

![E](./img/E.png)

Luego hacemos $XOR$ bit a bit del resultado anterior y la subllave $K_i$ y lo dividimos en 8 partes de 6 bits cada una. Es decir que $E(R_{i-1})\oplus K_i=B_1B_2B_3B_4B_5B_6$. Sabemos que cada $B_i$ es de 6 bits así que podemos escribirlo como $B_i=b_1b_2b_3b_4b_5b_6$. Cada $B_i$ será sustituido por un valor de 4 bits elegido a partir de una, así llamada, caja de sustitución o S-box. Hay una caja distinta por cada $B_i$. La manera de elegir el valor de la caja es tomar el renglón $b_1b_6+1$ y la columna $b_2b_3b_4b_5+1$, tomando en cuenta que el número estará en escrito en binario. Por ejemplo 101010 nos devolverá el cuarto renglón y sexta columna. Mostramos aquí las S-boxes.

![S-boxes](./img/S-boxes.png)

Por último concatenamos los resultados de las cajas y los pasamos por una última permutación P.

![P](./img/P.png)

Veamos este cifrado en acción. Comenzamos por completar el mensaje a encriptar.

In [4]:
m = b"Este es mi mensaje" #puedes insertar cualquier mensaje

while len(m) % 8 != 0: #completamos para que sea múltiplo de 64 bits
    m += b'\x00'
    
m

b'Este es mi mensaje\x00\x00\x00\x00\x00\x00'

Veremos el procesos con sólo uno de los bloques de 64 bits. Haremos la permutación y dividiremos en dos partes.

In [5]:
ip = [57, 49, 41, 33, 25, 17, 9,  1,
        59, 51, 43, 35, 27, 19, 11, 3,
        61, 53, 45, 37, 29, 21, 13, 5,
        63, 55, 47, 39, 31, 23, 15, 7,
        56, 48, 40, 32, 24, 16, 8,  0,
        58, 50, 42, 34, 26, 18, 10, 2,
        60, 52, 44, 36, 28, 20, 12, 4,
        62, 54, 46, 38, 30, 22, 14, 6]

mb = m[:8] #tomamos el primer bloque y lo convertimos a un arreglo de bits
l = len(mb) * 8
res = [0] * l
pos = 0
for ch in mb:
	i = 7
	while i >= 0:
		if ch & (1 << i) != 0:
			res[pos] = 1
		else:
			res[pos] = 0
		pos += 1
		i -= 1
mb = res

m0 = list(map(lambda x: mb[x], ip)) #aplicamos la permutación inicial

L = m0[:32] #dividimos en 2 partes
R = m0[32:]

Ahora veremos como hacer las iteraciones de los $L_i's$ y $R_i's$

In [6]:
expansion_table = [31,  0,  1,  2,  3,  4,
         3,  4,  5,  6,  7,  8,
         7,  8,  9, 10, 11, 12,
        11, 12, 13, 14, 15, 16,
        15, 16, 17, 18, 19, 20,
        19, 20, 21, 22, 23, 24,
        23, 24, 25, 26, 27, 28,
        27, 28, 29, 30, 31,  0]

p = [15, 6, 19, 20, 28, 11,
        27, 16, 0, 14, 22, 25,
        4, 17, 30, 9, 1, 7,
        23,13, 31, 26, 2, 8,
        18, 12, 29, 5, 21, 10,
        3, 24]

sbox = [
		# S1
		[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7,
		 0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8,
		 4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0,
		 15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13],

		# S2
		[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10,
		 3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5,
		 0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15,
		 13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9],

		# S3
		[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8,
		 13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1,
		 13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7,
		 1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12],

		# S4
		[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15,
		 13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9,
		 10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4,
		 3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14],

		# S5
		[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9,
		 14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6,
		 4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14,
		 11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3],

		# S6
		[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11,
		 10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8,
		 9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6,
		 4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13],

		# S7
		[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1,
		 13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6,
		 1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2,
		 6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12],

		# S8
		[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7,
		 1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2,
		 7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8,
		 2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11],]

for i in range(16): #hacemos 16 rondas

    tempR = R #guardamos temporalmente la R de i-1

    R = list(map(lambda x: R[x], expansion_table)) #expandimos la R

    R = list(map(lambda x, y: x ^ y, R, Kn[i])) #hacemos XOR con la subllave
    
    B = [R[:6], R[6:12], R[12:18], R[18:24], R[24:30], R[30:36], R[36:42], R[42:]]
    #dividimos en los pedazos de 6 bits
    
    Bn = [0] * 32
    
    for j in range(8): #Aplicamos las S-boxes
        
        m = (B[j][0] << 1) + B[j][5] #encontramos el valor a sustituir en las S-boxes
        n = (B[j][1] << 3) + (B[j][2] << 2) + (B[j][3] << 1) + B[j][4]
        v = sbox[j][(m << 4) + n]
        
        Bn[4 * j] = (v & 8) >> 3 #convertimos a bits y concatenamos el resultado de las cajas
        Bn[4 * j + 1] = (v & 4) >> 2
        Bn[4 * j + 2] = (v & 2) >> 1
        Bn[4 * j + 3] = v & 1
        
    R = list(map(lambda x: R[x], p)) #aplicamos la permutación P
    
    R = list(map(lambda x, y: x ^ y, R, L)) #hacemos XOR con L i-1 y obtenemos Ri
    
    L = tempR #igualamos el nuevo L al viejo R

Por último sabemos que debemos concatenar $R_i$ con $L_i$ y permutarlo con la inversa de la inicial.

In [7]:
fp = [39,  7, 47, 15, 55, 23, 63, 31,
        38,  6, 46, 14, 54, 22, 62, 30,
        37,  5, 45, 13, 53, 21, 61, 29,
        36,  4, 44, 12, 52, 20, 60, 28,
        35,  3, 43, 11, 51, 19, 59, 27,
        34,  2, 42, 10, 50, 18, 58, 26,
        33,  1, 41,  9, 49, 17, 57, 25,
        32,  0, 40,  8, 48, 16, 56, 24]

RL = R + L
final = list(map(lambda x: RL[x], fp))

ints = [] #pasamos a bytes el resultado
c=0
for i in range(len(final)):
    c += final[i] << (7 - (i % 8))
    if (i % 8) == 7:
        ints.append(c)
        c = 0
crypt = bytes(ints)
print (crypt)

b'\x8a\xaa\xca\x1f\xc3j\xb4\xeb'


El resultado puede consistir de letras o de bytes que no representan una letra en el estándar internacional. Estos bytes se verán como \x## donde el ## es el número que representa el byte escrito en hexadecimal.

## Desencriptar

El algoritmo para desencriptar es el mismo que el algoritmo para desencriptar. La diferencia es que para desencriptar debo usar las subllaves creadas en orden inverso a la versión de encriptado.

### Ejemplo

Usaremos el mismo bloque de mensaje usado arriba. Ya no explicaremos los pasos, a excepción de la inversión del orden de las llaves.

In [8]:
l = len(crypt) * 8
res = [0] * l
pos = 0
for ch in crypt:
    i = 7
    while i >= 0:
        if ch & (1 << i) != 0:
            res[pos] = 1
        else:
            res[pos] = 0
        pos += 1
        i -= 1
crp = list(map(lambda x: res[x], ip)) 
L = crp[:32] 
R = crp[32:]
for i in range(16): 
    tempR = R 
    R = list(map(lambda x: R[x], expansion_table))
    R = list(map(lambda x, y: x ^ y, R, Kn[15-i])) #Aquí revertimos el orden de las llaves
    B = [R[:6], R[6:12], R[12:18], R[18:24], R[24:30], R[30:36], R[36:42], R[42:]]
    Bn = [0] * 32
    for j in range(8): 
        m = (B[j][0] << 1) + B[j][5] 
        n = (B[j][1] << 3) + (B[j][2] << 2) + (B[j][3] << 1) + B[j][4]
        v = sbox[j][(m << 4) + n]
        Bn[4 * j] = (v & 8) >> 3 
        Bn[4 * j + 1] = (v & 4) >> 2
        Bn[4 * j + 2] = (v & 2) >> 1
        Bn[4 * j + 3] = v & 1
    R = list(map(lambda x: R[x], p)) 
    R = list(map(lambda x, y: x ^ y, R, L)) 
    L = tempR 
RL = R + L
final = list(map(lambda x: RL[x], fp))
ints = [] 
c=0
for i in range(len(final)):
    c += final[i] << (7 - (i % 8))
    if (i % 8) == 7:
        ints.append(c)
        c = 0
decrypt = bytes(ints)
print (decrypt)

b'Este es '


Como podemos ver, invertir el orden de las subllaves fue suficiente para devolver el mensaje desencriptado.